In [30]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import requests
from bs4 import BeautifulSoup
import scipy.io
import matplotlib.pyplot as plt
import matplotlib 
import pandas as pd
import numpy as np
import pickle
from time import sleep
import timeit
import sqlite3

%matplotlib inline

In [0]:
def req(msg, slp=0.2):
    """Make fault tolerant BGG server requests."""
    # Sleep to make sure you are not pinging the server to frequently
    sleep(slp)
    # Keep trying requests until status-code is 200
    status_code = 500
    while status_code != 200:
        sleep(slp)
        try:
            r = requests.get(msg)
            status_code = r.status_code
            # if status_code != 200:
                # print("Server Error! Response Code %i. Retrying..." % (r.status_code))
        except:
            # print("An exception has occurred, probably a momentory loss of connection. Waiting three seconds...")
            sleep(3)
    return r

In [33]:
r = requests.get("http://www.boardgamegeek.com/xmlapi2/user?name=Zazz&top=1")
soup = BeautifulSoup(r.text, "xml")  # Use the xml parser for API responses and the html_parser for scraping
print(r.status_code)  # 404 not found and the like. Hopefully 200!

200


In [0]:
def request(msg, slp=1):
    '''A wrapper to make robust https requests.'''
    status_code = 500  # Want to get a status-code of 200
    while status_code != 200:
        sleep(slp)  # Don't ping the server too often
        try:
            r = requests.get(msg)
            status_code = r.status_code
            if status_code != 200:
                print("Server Error! Response Code %i. Retrying..." % (r.status_code))
        except:
            print("An exception has occurred, probably a momentory loss of connection. Waiting one seconds...")
            sleep(1)
    return r

In [35]:
# Initialize a DF to hold all our scraped game info
df_all = pd.DataFrame(columns=["id", "name", "nrate", "pic_url"])
min_nrate = 1e5
npage = 1
i = 0

# Scrap successful pages in the results until we get down to games with < 1000 ratings each
while i <= 400:
    # Get full HTML for a specific page in the full listing of boardgames sorted by 
    r = request("https://boardgamegeek.com/browse/boardgame/page/%i?sort=numvoters&sortdir=desc" % (npage,))
    soup = BeautifulSoup(r.text, "html.parser")    
    
    # Get rows for the table listing all the games on this page
    table = soup.find_all("tr", attrs={"id": "row_"})  # Get list of all the rows (tags) in the list of games on this page
    df = pd.DataFrame(columns=["id", "name", "nrate", "pic_url"], index=range(len(table)))  # DF to hold this pages results
    
    # Loop through each row and pull out the info for that game
    for idx, row in enumerate(table):
        # Row may or may not start with a "boardgame rank" link, if YES then strip it
        links = row.find_all("a")
        if "name" in links[0].attrs.keys():
            del links[0]
        gamelink = links[1]  # Get the relative URL for the specific game
        gameid = int(gamelink["href"].split("/")[2])  # Get the game ID by parsing the relative URL
        gamename = gamelink.contents[0]  # Get the actual name of the game as the link contents
        imlink = links[0]  # Get the URL for the game thumbnail
        # thumbnail = imlink.contents[0]["src"]

        # ratings_str = row.find_all("td", attrs={"class": "collection_bggrating"})[2].contents[0]
        # nratings = int("".join(ratings_str.split()))

        df.iloc[idx, :] = [gameid]

    # Concatenate the results of this page to the master dataframe
    min_nrate = df["nrate"].min()  # The smallest number of ratings of any game on the page
    print("Page %i scraped, minimum number of ratings was %i" % (npage, min_nrate))
    df_all = pd.concat([df_all, df], axis=0)
    i =  len(df_all.index)
    print("Number of games scraped is %i" % i)
    npage += 1
    sleep(2) # Keep the BGG server happy.

Page 1 scraped, minimum number of ratings was 11
Number of games scraped is 100
Page 2 scraped, minimum number of ratings was 3
Number of games scraped is 200
Page 3 scraped, minimum number of ratings was 41
Number of games scraped is 300
Page 4 scraped, minimum number of ratings was 10
Number of games scraped is 400
Page 5 scraped, minimum number of ratings was 46
Number of games scraped is 500


In [0]:
df = df_all.copy()
# Reset the index since we concatenated a bunch of DFs with the same index into one DF
df.reset_index(inplace=True, drop=True)
# Write the DF to .csv for future use
df.to_csv("bgg_gamelist.csv", index=False, encoding="utf-8")

In [0]:
df_toy = df

In [0]:
#create a temporary dataframe with the right structure
#batch = current batch number 
#batch_size = number of raws in the dataframe
def df_temp(batch, batch_size):
  #  df = pd.DataFrame(columns=['type', 'name', 'yearpublished', 'minplayers', 'maxplayers',
  #      'playingtime', 'minplaytime', 'maxplaytime', 'minage', 'usersrated',
  #      'average',  'owned',
  #      'trading', 'wanting', 'wishing', 'numcomments',
  #       'numweights', 'averageweight', 'links'], index=range(batch*batch_size, batch_size+batch*batch_size))
   df = pd.DataFrame(index=range(batch*batch_size, batch_size+batch*batch_size))
  
   return df   

In [0]:
def slice_ids(df, batch, batch_size):
  added_index = df.loc[(df.index >= batch*batch_size) & (df_toy.index < batch_size+batch*batch_size), ['id']]
  return added_index

In [0]:
#requests current batch of xml's based on df's ids
#should have "id" column
def request_df_xml(df):
  id_strs = [str(gid) for gid in df["id"]]
  gameids = ",".join(id_strs)
  sleep(1.5)  # Keep the server happy
  r = request("http://www.boardgamegeek.com/xmlapi2/thing?id=%s&stats=1" % gameids)
  return r

In [0]:
def add_values_xml_to_df(r, df):
  soup = BeautifulSoup(r.text, "xml")
  items = soup.find_all('item')
  # print(len(items))
  gtype = soup.find_all("type")
  name = soup.find_all("name", {"type":"primary"})
  yearpublished = soup.find_all ("yearpublished")
  minplayers = soup.find_all ("minplayers")
  maxplayers = soup.find_all ("maxplayers")
  playingtime = soup.find_all ("playingtime")
  minplaytime = soup.find_all ("minplaytime")
  maxplaytime = soup.find_all ("maxplaytime")
  minage = soup.find_all("minage")
  users_rated = soup.find_all("usersrated")
  average_rating = soup.find_all("average")
  total_owners = soup.find_all ("owned")
  total_traders = soup.find_all("trading")
  total_wanters = soup.find_all("wanting")
  total_wishers = soup.find_all("wishing")
  total_comments = soup.find_all("numcomments")
  total_weights = soup.find_all("numweights")
  average_weight = soup.find_all ("averageweight")
  links= []
  for item in items: 
    # link.append(items[item])
    links.append(item.findChildren("link", recursive = "False"))
    
  # print(len(links))

In [0]:
def create_temp_df(r):
  soup = BeautifulSoup(r.text, "xml")
  list_all_games = soup.find_all('item')
  items = soup.find_all('item')
  data  = {
      'gameid': [],
      'gametype':[],
      'name': [],
      'yearpublished': [],
      'minplayers': [],
      "maxplayers": [],
      'playingtime': [],
      'minplaytime': [],
      'maxplaytime': [],
      'minage': [],
      'usersrated': [],
      'average': [],
      'owned': [],
      'trading': [],
      'wanting': [],
      'wishing': [],
      'numcomments': [],
      'numweights': [],
      'averageweight': [],
      'categories':[],
      'mechanics':[],
      'designers':[],
      'ranksubtypes': [],
      'rankfamilies': [],
      'families':[],
      'expansions':[],
      'implementations':[],
      'publishers': []
  }
  for item in items: 
    # link.append(items[item])
    gameid = item['id']
    gametype = item['type']
    name = item.findChild("name", {"type":"primary"})['value']
    yearpublished = item.findChild("yearpublished")['value']
    minplayers = item.findChild("minplayers")['value']
    maxplayers = item.findChild("maxplayers")['value']
    playingtime = item.findChild("playingtime")['value']
    minplaytime = item.findChild("minplaytime")['value']
    maxplaytime = item.findChild("maxplaytime")['value']
    minage = item.findChild("minage")['value']
    usersrated = item.findChild("usersrated")['value']
    average = item.findChild("average")['value']
    owned = item.findChild("owned")['value']
    trading = item.findChild("trading")['value']
    wanting = item.findChild("wanting")['value']
    wishing = item.findChild("wishing")['value']
    numcomments = item.findChild("numcomments")['value']
    numweights = item.findChild("numweights")['value']
    averageweight = item.findChild("averageweight")['value']
    categories = item.findChildren("link", {"type": "boardgamecategory"})
    mechanics = item.findChildren("link", {"type": "boardgamemechanic"})
    designers = item.findChildren("link", {"type": "boardgamedesigner"})
    subtypes = item.findChildren("rank",{"type": "subtype"})
    rankfamilies = item.findChildren("rank", {"type": "family"})
    families = item.findChildren("link", {"type":"boardgamefamily"})
    expansions = item.findChildren("link", {"type":"boardgameexpansion"})
    implementations = item.findChildren("link", {"type": "boardgameimplementation"})
    publishers = item.findChildren("link", {"type":"boardgamepublisher"})

    # print(categories)
    categories_list = []
    mechanics_list = []
    designers_list = []
    subtypes_list = []
    rankfamilies_list = []
    families_list = []
    expansions_list = []
    implementations_list = []
    publishers_list = []
    # НЕ ПОНЯТНО, КАКИЕ ИМЕННО VALUES ВЫТЯГИВАТЬ, IDS ИЛИ NAMES ИЛИ И ТО И ТО :((
    for category in categories:
      categories_list.append(category['value'])
    for mechanic in mechanics:
      mechanics_list.append(mechanic['value'])
    for designer in designers:
      designers_list.append(designer['value'])
    for subtype in subtypes:
      subtypes_list.append(subtype['name'])   
    for family in rankfamilies:
      rankfamilies_list.append(family['name'])  
    for family in families:
      families_list.append(family['value'])
    for expansion in expansions:
      expansions_list.append(expansion['id'])   
    for implementation in implementations: 
      implementations_list.append(implementation['id'])   
    for publisher in publishers: 
      publishers_list.append(publisher['id'])    
    data['gameid'].append(gameid)
    data['gametype'].append(gametype)
    data['name'].append(name)
    data['yearpublished'].append(yearpublished)
    data['minplayers'].append(minplayers)
    data['maxplayers'].append(maxplayers)
    data['playingtime'].append(playingtime)
    data['minplaytime'].append(minplaytime)
    data['maxplaytime'].append(maxplaytime)
    data['minage'].append(minage)
    data['usersrated'].append(usersrated)
    data['average'].append(average)
    data['owned'].append(owned)
    data['trading'].append(trading)
    data['wanting'].append(wanting)
    data['wishing'].append(wishing)
    data['numcomments'].append(numcomments)
    data['numweights'].append(numweights)
    data['averageweight'].append(averageweight)
    data['categories'].append(categories_list if categories_list else '') #is it safe to use empty string as a placeholder?
    data['mechanics'].append(mechanics_list if mechanics_list else '')
    data['designers'].append(designers_list if designers_list else '')
    data['ranksubtypes'].append(subtypes_list if subtypes_list else '')
    data['rankfamilies'].append(rankfamilies_list if rankfamilies_list else '')
    data['families'].append(families_list if families_list else '')
    data['expansions'].append(expansions_list if expansions_list else '')
    data['implementations'].append(implementations_list if implementations_list else '')
    data['publishers'].append(publishers_list if publishers_list else '')

  table = pd.DataFrame.from_dict(data)
  return table

In [43]:
# Gathering all METADATA from all games in toy data set
#############################################################
# Get the metadata for all games, but do it in chunks of 250 games
df_meta_all = pd.DataFrame()
batch_size = 250
for batch, group in df_toy.groupby(np.arange(len(df_toy))//batch_size):
      print("Processed ratings for batch #%i of games." % (batch))
      print(group)
   
    # Initialize a DF to hold all the responses for this chunk of games
      df_meta = df_temp(batch, batch_size)
      # Set the next chunk of the DF "gameid" column using the list of game IDs
      added_index = slice_ids(df_toy, batch, batch_size)
      df_meta = pd.concat([df_meta, added_index], axis = 1)
      r = request_df_xml(df_meta)
      temp = create_temp_df(r)
      df_meta_all = df_meta_all.append(temp, ignore_index=True, sort = False)
# print(df_meta_all)      
df_meta_all.to_csv('/content/gdrive/My Drive/Thesis/test.csv')

 

Processed ratings for batch #0 of games.
         id    name   nrate pic_url
0        13      13      13      13
1       822     822     822     822
2     30549   30549   30549   30549
3     68448   68448   68448   68448
4     36218   36218   36218   36218
..      ...     ...     ...     ...
245     483     483     483     483
246  115746  115746  115746  115746
247  155362  155362  155362  155362
248     811     811     811     811
249      41      41      41      41

[250 rows x 4 columns]
Processed ratings for batch #1 of games.
         id    name   nrate pic_url
250  172818  172818  172818  172818
251  224517  224517  224517  224517
252  236457  236457  236457  236457
253   46213   46213   46213   46213
254   66188   66188   66188   66188
..      ...     ...     ...     ...
495    2389    2389    2389    2389
496  232043  232043  232043  232043
497   36932   36932   36932   36932
498   82222   82222   82222   82222
499  179275  179275  179275  179275

[250 rows x 4 columns]
     g